This project was severely limited by my time crunch, and I am disappointed that I can't present my best possible work to you. I was at a hackathon on Friday and Saturday nights, and came back at midnight on Sunday due to the late awards ceremony. Hence, I only had Monday night and one night the week prior to work on this. I spent the night learning everything I now know about data science and the statistical algorithms, and now have at least an elementary understanding of most of their functioning. I hope you see, even though this may not be my best work yet, that I am incredibly commmited and motivated to learn on the go, and I promise to work as hard as this, if not more, over the 4 months of the summer! 

This is my solution for Question 1. I started by making the imports:

In [ ]:
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, scale
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
from sklearn.metrics import classification_report

Define a class called Prediction that will act as my predictor.

In [ ]:
class Prediction:
	def __init__(self, filepath):
		self.data = pandas.read_csv(filepath)
        self.model = 0
		self.traindata = 0
		self.testdata = 0
		self.successes = self.data.loc[data['state']=='successful']
		self.fails = self.data.loc[data['state']=='failed']
		self.targetTrain = 0
		self.targetTest = 0        

I want to remove all rows that contain anything in Row 14, manually titled 'exists', as this implies that the data is messed up as it is out of order. I found this by first manually inspecting the data for some time. I do all of the preprocessing in the init function. 

In [ ]:
		badrows = self.data[self.data['exists'].notnull()]
		self.data = pd.concat([self.data, badrows, badrows]).drop_duplicates(keep=False)

We want to remove the rows that don't contain a type datetime in them. in the columns launched and deadline. They have type int instead, which was found by listing all the types in the two columns. We can use iterrows() even though it is slow, as I wrote to the csv file already with the fixed information.

In [ ]:
		indicesToDelete = []
		indicesToKeep = []

		for index, row in data.iterrows():
			try:
				int(row['deadline'])
				indicesToDelete.append(row[index])
			except:
				indicesToKeep.append(index)
		self.data = self.data.drop(indicesToDelete, axis=0)
        
        

Convert the datetime in deadline and launched column to datetime object that python can handle.

In [ ]:
	def convertToTime(self, date):
            dateupdate = datetime.strptime(date, '%Y-%m-%d %H:%M')
            return dateupdate
        #In the init
        self.data = self.data['launched'].apply(convertToTime)
		self.data = self.data['deadline'].apply(convertToTime)

Convert numbers in the two number columns floats

In [ ]:
		self.data["goal"] = self.data['goal'].astype('float')
		self.data["pledged"] = self.data['pledged'].astype('float')


Make a new column for time difference. This is one of the features I will be seeing if it is relevant. This is just defined as the length between the deadline and when it was launched.

In [ ]:
		self.data['timeDifference'] = (self.data['deadline'] - data['launched'])
		self.data['timeDifference'] = self.data.timeDifference.dt.total_seconds()

Next, I want to convert my target, the state, to a boolean (however I am using 1 and 0 so that the algorithms can understand this). Then, I want to delete all entries that are not success/fail.

In [ ]:
	def convertToBool(self, item):
        if item=='failed':
            return 0
        if item=='successful':
            return 1
        else:
            return 2
        
        #In the init
        self.data['state'] = self.data['state'].apply(convertToBool)
		self.data = self.data.drop(self.data[self.data.state == 2].index)

Now I want to play with the launched/deadline fields a little more and see how the month of the deadline and month it was launched has an impact on success/fail. 

In [ ]:
		#Want to convert datetime to month in new column
		self.data['month_launched'] = (self.data.launched.dt.month)
		self.data['month_deadline'] = (self.data.deadline.dt.month)


Now I am in a position to manually assess the relevance of specific features. I define genBarGraph() to generate bar graphs of success/fail ratios for a specific feature.

In [ ]:
	def genBarGraph(self, column):
		#Edit a little to generate non-ratio graphs
		labels = self.fails[column].value_counts().keys().tolist()
		countsfails = self.fails[column].value_counts().tolist()
		countssuccesses = self.successes[column].value_counts().tolist()

		ratios = []
		for i in range(len(labels)):
			ratios.append(countsfails[i]/countssuccesses[i])

		index = np.arange(len(labels))
		plt.xticks(index, labels, fontsize=5, rotation=30)
		plt.xlabel(column, fontsize=5)
		plt.ylabel('Ratio', fontsize=5)
		plt.title('Success/Fail ratio for each' + column)
		plt.bar(index, ratios)
		plt.show()

		return 1


In [ ]:
Using this function, I generated the following graphs that are shown. 

![title](graphs/categoryRatio.png) ![title](graphs/countryRatio.png) ![title](graphs/deadlineMonthRatio.png) ![title](graphs/mainCategoryRatio.png) ![title](graphs/monthLaunched.png) ![title](graphs/currencyRatio.png) 

From these graphs, I can determine that the category, country, the currency and the main category have a wide variation the success/fail ratios, therefore we will delete deadline month and launched month as they are not relevant. Instead of deleting them explicitly, I just will not encode them and them they will get filtered out later. Now, I can start encoding the discrete data. I defined a function genEncoder that uses scikit's LabelEncoder() and OneHotEncoder() libraries to create encodings. 

In [ ]:
	def genEncoder(self, data, columns):
		for i in columns:
			le_column = LabelEncoder()
			ohe_column = OneHotEncoder()

			self.data[i+'Encoded'] = le_column.fit_transform(self.data[i].fillna('0'))

			columnBinaryArray = ohe_column.fit_transform(self.data[i+'Encoded'].values.reshape(-1,1)).toarray()
			categoryOneHot = pd.DataFrame(columnBinaryArray, columns = [i+ '_' + str(int(k)) for k in range(columnBinaryArray.shape[1])])

			self.data = pd.concat([self.data, categoryOneHot], axis=1)

		return data


Now, in the init function, I can just call this function as follows:

In [ ]:
		columns = ['country', 'main_category','currency',  'category']
		self.data = self.genEncoder(self.data, columns)

It is now time to vectorize the title. Here, I explored a few options such as word2vec and gloVe, along with potentially doing sentiment analysis. However, my argument for not doing those is that the titles of these projects are so specific and wildly descriptive, that those two algorithms might not be the best way to proceed. Titles may contain made-up words that would never appear again, and so would make the vectorizing not very useful for comparisons. Therefore, I vectorize the titles simply through comparing their title lengths.

In [ ]:
		self.data['titleLength'] = self.data['name'].apply(lambda x: len(str(x)))

Now, it is time to scale the data. The reason why it's imperative that we do this is because some of the algorithms being used later need scaled data, and give weird results otherwise. This code block also filters out all those data fields that are not type float64 (such as month launched and month deadline)

In [ ]:
		for i in self.data.keys():
			if self.data[i].dtype == 'float64' and i!='ID' and i!='state':
				self.data[i] = scale(self.data[i])             

Now that we have scaled our data, we can start generating test data. I used the traintestsplit function from scikit. However, before I can get a dataset, I need to make sure that the train set created is balanced, this means that it has around an even set of success/fail.

In [ ]:
	def genTrainTestData(self):
		#Delete all unncessary, unvectorized columns
		for i in data.keys():
			if not data[i].dtype =='float':
			data = data.drop(i, axis=1)
            
		#count = self.data['state'].value_counts() #Done to check how many success/fails. 1.48 ratio found.
		#It is important to check this as we need to make sure our data is balanced.
		#A ratio of 1.5 is acceptable, especially given our data and after consulting with internet resources
		
		features = data.values[:, :data.columns.get_loc("state")]
		target = data.values[:, data.columns.get_loc("state")]

		successfailratio = 10

		while successfailratio > 1.5:
			self.featureTrain, self.featureTest, self.targetTrain, self.targetTest = train_test_split(featuers,target, test_size=0.2)
			counts = self.train['state'].value_counts().tolist()
			successfailratio = counts[0]/counts[1]

		#Remove the NaN values from each array. It would be better if I replace these with the means of each column instead of just 0.
		self.featureTrain = np.nan_to_num(self.featureTrain)
		self.featureTest = np.nan_to_num(self.featureTest)
		self.targetTrain = np.nan_to_num(self.targetTrain)
		self.targetTrain = np.nan_to_num(self.targetTrain)

		return 1

Now that we have finally generated our test and training data, we can start implementing some machine learning algorithms to generate a model. I will start by implementing the Gaussian Naive-Bayes Classification algorithm. Though it assumes independence, we can use it as a base classifier for our other models.

In [ ]:
	def gaussianNaiveBayes(self):
		model = GaussianNB()
		model.fit(self.featureTrain, self.targetTrain)

		prediction = model.predict(self.targetTest)
		pred_accuracy = classification_report(self.targetTest, prediction)
    
        self.model = model
		return pred_accuracy

This model gives us a precision accuracy of 0.65, which is better than 0.5 so it is already a decent model to use.

![title](results/naivebayes.png)

Now we can define the function for logistic regression. Logistic regression is also not the best algorithm to use, but we will use it here as it will allow me to compare it with other algorithms. It fits our desires well as it is a binary classification algoritns, and works well with high dimensions. We also removed the date of the deadline and launch, which may have been redundant pieces of information that would make the algorithm less efficient. One thing for the future may be to see the similarities between country/currency and other similar fields as this also fogs the algorithm.

In [ ]:
	def logisticRegression(self):
		model = LogisticRegression()

		model.fit(self.featureTrain, self.targetTrain)

		prediction = model.predict(self.targetTest)
		pred_accuracy = classification_report(self.targetTest, prediction)

		return pred_accuracy

For the results of this logistic regression, I actually ran into an error and was unable to find the solution to the bug. I didn't have much time to work on this project, and so I moved on and decided to come back to this later if I had time. Sorry! :(

We can also define a function for a decision tree classification algorithm. The reason I included this is that although this is much worse than random forests as they are very prone to overfitting due to high bias, I wanted to learn how random forests work, and this seemed like a preliminary step to that. However, the results I got from this model weren't very fruitful.

In [ ]:
	def decisionTreeClassifier(self):
		#Use the roc-auc scoring 
		scoring = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)}
		clf = GridSearchCV(DecisionTreeClassifier(random_state=15), param_grid={'min_samples_split': range(2, 253, 10)}, scoring=scoring, cv=5, refit='AUC', return_train_score=True)
		
		clf.fit(train, target_train)
		optimalSplit = clf.best_params_

		#Gives an optimalSplit of 242

		model = DecisionTreeClassifier(random_state=clf.best_params_)
		model.fit(self.featureTrain, self.targetTrain)

		prediction = model.predict(self.targetTest)
		pred_accuracy = classification_report(self.targetTest, prediction)

		return pred_accuracy
    

We also need to define a function called predict to actually predict new data. Error checking is important here, however no time :()

In [ ]:
def Predict(self, data):
    ynew = self.model.predict(data)
    print("X=%s, Predicted=%s" % (data, ynew))
    return 1

![title](results/decisiontree.png)

As you can see, this shows that this decision tree is an almost perfect fit of the data. I'm still slightly confused as to why this is happening, as I did split my dataset so if this was a case of overfitting, I would have a very low accuracy.

I also had plans to test other algorithms, namely Support Vector Machines, kNN-clustering and random forests, however due to time limitations I could not. The best algorithm I had right now was the naive-Bayes algorithm, so I set it as the current best model. 